In [1]:
import os
import pandas as pd
import geopandas as gpd
from scipy import interpolate
from scipy import signal
import ee
import json
from collections import Counter
import datetime
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
# read file using geopandas
gpf = gpd.read_file('merged_irr_final/merged_irr_final/sy_merged_sowing.shp')
gpf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
def get_collection(start_date, end_date, roi, collection='COPERNICUS/S2_SR'):
    """ Filters an ee.ImageCollection. COPERNICUS/S2_SR (Sentinel 2) is the default collection.

        Args:
            start_date (str): start date of the filter (YYYY-MM-DD)
            end_date (str): end date of the filter (YYYY-MM-DD)
            roi (ee.geometry, ee.FeatureCollection): the geometry to intersect
            collection (str): the desired dataset (https://developers.google.com/earth-engine/datasets)

        Returns:
            An ee.ImageCollection
    """
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    filtered_collection = ee.ImageCollection(collection) \
        .filterBounds(roi) \
        .filterDate(start, end)
    return filtered_collection

In [5]:
def calc_s2_vegetation_index(image):
    """ Calculates the vegetation indeces.

        Args:
            image: ee.Image

        Returns:
            Adds the NDVI image to the ee.Image
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [6]:
def cloud_mask_probability(image, max_cloud_prob=65):
    """ Applies the S2 cloud probability mask.

        Args:
            image (ee.Image): ee.Image
            max_cloud_prob: The cloud probability threshold

        Returns:
            Updates the cloud mask in the ee.Image
    """
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(max_cloud_prob)
    return image.updateMask(isNotCloud)

In [7]:
def mask_edges(image):
    """ Sometimes, the mask for the 10m bands does not exclude bad pixels at the image edges.
        Therefore, it's necessary to apply the masks for the 20m and 60m bands as well.
        Reference: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY#description

        Args:
            image (ee.Image): ee.Image to which the mask is to be applied

        Returns:
            Updates the cloud mask in the ee.Image

    """
    return image.updateMask(image.select('B8A').mask().updateMask(image.select('B9').mask()))

In [8]:
def cloud_mask_qa(image):
    """ Applies the S2 QA mask.

Args:
    image (ee.Image): ee.Image

Returns:
    Updates the cloud mask in the ee.Image
    """
    cloudShadowBitMask = (1 << 10)
    cloudsBitMask = (1 << 11)
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [29]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=10,
                                  crs='EPSG:3857',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
    def reduce_region_function(img):
        """Applies the ee.Image.reduceRegion() method.
            Reference: https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair
            Args:
                img:
                    An ee.Image to reduce to a statistic by region.

            Returns:
                An ee.Feature that contains properties representing the image region
                reduction results per band and the image timestamp formatted as
                milliseconds from Unix epoch (included to enable time series plotting)
        """
        try:
            stat = img.reduceRegion(
                reducer=reducer,
                geometry=geometry,#.transform(crs, 0.001)
                scale=scale,
                crs=crs)
                # bestEffort=bestEffort,
                # maxPixels=maxPixels,
                # tileScale=tileScale)
            millis = img.get('system:time_start')
            date_time = ee.Date(millis).format('YYYY-MM-dd HH:mm:ss')

            return ee.Feature(geometry, stat).set({ 
                'millis': millis,
                'datetime': date_time,
                'id': img.id()
                }).setGeometry(None)
        except ee.EEException as e:
            # Handle the exception (e.g., print an error message)
            print(f"Error processing image: {e}")
            return None

    return reduce_region_function

In [10]:
def fc_to_dict(fc):
    """ Transfers the properties of the feature to a dictionary.
        Reference: https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair

        Args:
            fc (ee.FeatureCollection): ee.FeatureCollection

        Returns:
            A dictionary
    """
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [11]:
# process input data, get the sowing date and calculate start and end date for the image collection, also calculate irrigation dates
gpf['sowing_dat'] = pd.to_datetime(gpf['sowing'], format='%d-%m-%y').dt.strftime('%Y-%m-%d')
# gpf.head()
# start date is 30 days before sowing date, end date is 150 days after sowing date
gpf['start_date'] = pd.to_datetime(gpf['sowing']) - pd.DateOffset(days=30)
gpf['end_date'] = pd.to_datetime(gpf['sowing']) + pd.DateOffset(days=150)
gpf['sowing_dat'] = pd.to_datetime(gpf['sowing_dat'])
gpf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   plot1       399 non-null    object        
 1   shape_P1    399 non-null    object        
 2   plot2       399 non-null    object        
 3   shape_P2    399 non-null    object        
 4   plot3       399 non-null    object        
 5   shape_P3    399 non-null    object        
 6   sowing      399 non-null    object        
 7   Origin      399 non-null    object        
 8   group       399 non-null    int64         
 9   sowing_dat  399 non-null    datetime64[ns]
 10  mean_ndvi_  399 non-null    float64       
 11  geometry    399 non-null    geometry      
 12  start_date  399 non-null    datetime64[ns]
 13  end_date    399 non-null    datetime64[ns]
dtypes: datetime64[ns](3), float64(1), geometry(1), int64(1), object(8)
memory usage: 43.8+ KB


/var/folders/s0/1w918fdn5qqbxjb9dd551mbw0000gn/T/ipykernel_88157/1575401901.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gpf['start_date'] = pd.to_datetime(gpf['sowing']) - pd.DateOffset(days=30)
/var/folders/s0/1w918fdn5qqbxjb9dd551mbw0000gn/T/ipykernel_88157/1575401901.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gpf['end_date'] = pd.to_datetime(gpf['sowing']) + pd.DateOffset(days=150)


In [14]:
def extract_DAS(row, plot_column):
    das_values = map(int, re.findall(r'(\d+) DAS', row[plot_column]))
    if not das_values:
        return []
    sowing_date = row['sowing_dat']
    irr_dates = [sowing_date + pd.DateOffset(days=das) for das in das_values]
    irr_dates = [date.strftime('%Y-%m-%d') for date in irr_dates]
    # print(type(irr_dates))
    return list(irr_dates)

In [16]:
for index, row in gpf.iterrows():
    if row['Origin'] == 'p1':
        irr_dates1 = extract_DAS(row, 'plot1')
        gpf.at[index, 'irr_dates'] = irr_dates1
    elif row['Origin'] == 'p2':
        irr_dates2 = extract_DAS(row, 'plot2')
        gpf.at[index, 'irr_dates'] = irr_dates2
    elif row['Origin'] == 'p3':
        irr_dates3 = extract_DAS(row, 'plot3')
        gpf.at[index, 'irr_dates'] = irr_dates3

In [17]:
gpf.sample(5)

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,sowing,Origin,group,sowing_dat,mean_ndvi_,geometry,start_date,end_date,irr_dates
68,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.91871735453606 25.292777893636192, 83.91846...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",83.91849305480719 25.29297068756803,"CT with 2 irrigations (21 DAS, 65 DAS)",83.91827914863825 25.293012217038385,08-12-22,p1,23,2022-12-08,0.053399,"POLYGON ((83.91872 25.29278, 83.91847 25.29284...",2022-07-13,2023-01-09,"[2022-12-29, 2023-02-11, 2023-03-03, 2023-03-23]"
111,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","84.24664691090584 25.581626826994828, 84.24646...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","84.24665596336126 25.582019656103807, 84.24645...","CT with 2 irrigations (21 DAS, 65 DAS)","84.24589421600103 25.582068646248683, 84.24572...",20-12-22,p1,39,2022-12-20,0.011416,"POLYGON ((84.24665 25.58163, 84.24646 25.58164...",2022-11-20,2023-05-19,"[2023-01-10, 2023-02-23, 2023-04-04]"
161,"CT with 2 irrigations (21 DAS, 65 DAS)","84.46009978652 25.09330684352194, 84.460283182...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",84.46025636047125 25.093155634587067,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",84.46002166718245 25.09312010956923,12-12-22,p1,55,2022-12-12,0.103586,"POLYGON ((84.4601 25.09331, 84.46028 25.09323,...",2022-11-12,2023-05-11,"[2023-01-02, 2023-02-15]"
180,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",84.6472117 26.7583233,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","84.64762039500005 26.75801948800006, 84.647377...","CT with 2 irrigations (21 DAS, 65 DAS)",84.64705009013414 26.75812905876344,20-11-22,p2,64,2022-11-20,0.086729,"POLYGON ((84.64762 26.75802, 84.64738 26.75807...",2022-10-21,2023-04-19,"[2022-12-11, 2023-01-24, 2023-03-05]"
146,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","84.38340503722429 26.779088930649074, 84.38330...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","84.38316062092781 26.77900542093328, 84.383294...","CT with 2 irrigations (21 DAS, 65 DAS)","84.38311904668808 26.779331677180256, 84.38282...",18-11-22,p1,50,2022-11-18,0.011353,"POLYGON ((84.38341 26.77909, 84.3833 26.77893,...",2022-10-19,2023-04-17,"[2022-12-09, 2023-01-22, 2023-02-11, 2023-03-03]"


In [21]:
type(gpf['geometry'][0])

shapely.geometry.polygon.Polygon

In [23]:
# convert shapely geometry to ee.Geometry
g = json.loads(gpf['geometry'].to_json())
coords = g['features'][0]['geometry']['coordinates']
geom = ee.Geometry.Polygon(coords)
gpf['geometry'] = geom

/var/folders/s0/1w918fdn5qqbxjb9dd551mbw0000gn/T/ipykernel_88157/464729997.py:5: UserWarning: Geometry column does not contain geometry.
  gpf['geometry'] = geom


In [24]:
type(gpf['geometry'][0])

ee.geometry.Geometry

In [25]:
test = gpf.iloc[0]
test

plot1         CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...
shape_P1      83.27556654810905 26.51913650693408, 83.275912...
plot2          CT with 3 irrigations  (21 DAS, 65 DAS, 105 DAS)
shape_P2      83.275759331882 26.518783706222347, 83.2762552...
plot3                    CT with 2 irrigations (21 DAS, 65 DAS)
shape_P3      83.27629443258047 26.51849600483952, 83.276851...
sowing                                                 21-11-22
Origin                                                       p2
group                                                         0
sowing_dat                                  2022-11-21 00:00:00
mean_ndvi_                                             0.070139
geometry      ee.Geometry({\n  "functionInvocationValue": {\...
start_date                                  2022-10-22 00:00:00
end_date                                    2023-04-20 00:00:00
irr_dates                  [2022-12-12, 2023-01-25, 2023-03-06]
Name: 0, dtype: object

In [33]:
def get_img_collection(row):
        s2_sr = get_collection(row['start_date'], row['end_date'], row['geometry'], collection="COPERNICUS/S2_SR_HARMONIZED")
        s2_sr = s2_sr.map(calc_s2_vegetation_index)
        s2_sr.first().getInfo()
        s2_cloud_prob = get_collection(row['start_date'], row['end_date'], row['geometry'], collection="COPERNICUS/S2_CLOUD_PROBABILITY")
        s2_sr_with_cloud_mask = ee.Join.saveFirst('cloud_mask').apply(
                primary=s2_sr.map(mask_edges).map(cloud_mask_qa),
                secondary=s2_cloud_prob,
                condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))
        s2_sr_with_cloud_mask.first().getInfo()
        imgCollection = ee.ImageCollection(s2_sr_with_cloud_mask).map(cloud_mask_probability)
        # imgCollection.first().getInfo()
        reducer = ee.Reducer.mean()
        reduce_function = create_reduce_region_function(geometry=row['geometry'], reducer=reducer, scale=10)
        roi_stat = ee.FeatureCollection(imgCollection.map(reduce_function))
        df_stats = ee.data.computeFeatures({'expression': roi_stat,
                                        'fileFormat': 'PANDAS_DATAFRAME'
                                        })
        return df_stats

In [35]:
df_stats = get_img_collection(test)
df_stats

,geo,AOT,B1,B11,B12,B2,B3,B4,B5,B6,...,QA20,QA60,SCL,TCI_B,TCI_G,TCI_R,WVP,datetime,id,millis
0,None,411.849764,548.304402,1641.503705,832.695823,614.670222,818.281945,608.418145,1171.391085,2437.411745,...,0.0,0.0,4.0,63.041433,83.451493,62.007523,2490.663934,2022-10-22 05:11:27,20221022T045841_20221022T050319_T44RQQ,1666415487217
1,None,576.000000,190.387042,1654.690209,852.864586,379.187065,615.525488,450.107231,1092.936784,2414.640243,...,0.0,0.0,4.0,38.793173,63.020997,46.335953,1892.798338,2022-10-27 05:11:24,20221027T045909_20221027T050314_T44RQQ,1666847484394
2,None,370.315293,430.250168,1700.523916,897.312149,477.156748,726.723557,609.632046,1224.600719,2225.735235,...,0.0,0.0,4.0,48.664945,74.435661,63.202111,2072.689311,2022-11-01 05:11:29,20221101T045941_20221101T045941_T44RQQ,1667279489294
3,None,510.000000,516.563665,1894.449360,1071.829216,710.834381,903.462048,832.401639,1388.611947,2120.159443,...,0.0,0.0,5.0,72.878060,92.166405,85.310240,2124.737593,2022-11-06 05:11:23,20221106T045959_20221106T050606_T44RQQ,1667711483591
4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-11 05:11:28,20221111T050031_20221111T050905_T44RQQ,1668143488800
5,None,324.000000,600.147541,2087.007523,1619.363126,725.067370,802.737143,945.329778,1195.345497,1373.655850,...,0.0,0.0,5.0,74.106894,81.862565,96.222322,1679.982933,2022-11-16 05:11:25,20221116T050049_20221116T050837_T44RQQ,1668575485032
6,None,426.000000,522.954413,3016.980463,2581.752751,794.989782,1113.810465,1483.265439,1692.451718,1766.666180,...,0.0,0.0,5.0,81.621716,113.590164,151.510555,1197.567932,2022-11-21 05:11:25,20221121T050121_20221121T050115_T44RQQ,1669007485844
7,None,360.000000,673.703571,3206.850999,2999.295307,1042.052886,1309.860094,1637.765776,1828.527397,1888.213788,...,0.0,0.0,5.0,106.039636,133.388951,167.008758,1362.859757,2022-11-26 05:11:24,20221126T050129_20221126T051021_T44RQQ,1669439484538
8,None,312.000000,820.161689,3129.463283,2778.657310,1008.418482,1315.312598,1583.117898,1826.823939,1966.643948,...,0.0,0.0,5.0,103.149113,134.462048,161.566472,1006.239501,2022-12-01 05:11:25,20221201T050151_20221201T050150_T44RQQ,1669871485493
9,None,372.000000,837.998653,3113.186616,2820.249495,1145.449360,1392.347631,1609.450483,1851.788345,2048.239951,...,0.0,0.0,5.0,116.934539,141.805974,163.744554,1067.413092,2022-12-06 05:11:22,20221206T050159_20221206T050158_T44RQQ,1670303482282
